In [12]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import re

# 定義清理商品名稱的函數
def clean_title(title):
    clean_title = re.sub(r'\(.*?\)', '', title).strip()
    return clean_title

# 爬取創捷國際的數據
def fetch_data_from_teksource():
    products = []

    def scrape_page(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        product_sections = soup.find_all('div', class_='tp-sec tp-s2')
        for section in product_sections:
            product_link = section.find('a')
            if product_link:
                title = product_link.find('h2', class_='title h6')
                if title:
                    title = title.text.strip()
                    price_section = section.find_next_sibling('div', class_='tp-sec tp-s3')
                    if price_section:
                        price_div = price_section.find('div', class_='tp-inner tp-i1')
                        if price_div:
                            price_elem = price_div.find('span', class_='price h5 color1')
                            if price_elem:
                                price = price_elem.text.strip().replace(',', '')  # 去掉逗號
                                products.append((title, float(price[1:]), '創捷國際'))  # 將價格轉為浮點數並去掉前面的貨幣符號
                            else:
                                products.append((title, 0, '創捷國際'))  # 沒有找到價格，設為0

    base_urls = [
        'https://www.teksource.com.tw/products/%E5%95%86%E7%94%A8%E7%AD%86%E9%9B%BB-i.840.',
        'https://www.teksource.com.tw/products/%E5%AE%B6%E7%94%A8%E7%AD%86%E9%9B%BB-i.841.',
        'https://www.teksource.com.tw/products/%E9%9B%BB%E7%AB%B6%E7%AD%86%E9%9B%BB-i.842.',
        'https://www.teksource.com.tw/products/Apple%20%E7%AD%86%E8%A8%98%E5%9E%8B%E9%9B%BB%E8%85%A6-i.116.'
    ]

    for base_url in base_urls:
        for i in range(0, 120, 12):
            page_url = base_url + str(i)
            scrape_page(page_url)

    return products

# 爬取台灣大哥大的數據
def fetch_data_from_taiwanmobile():
    products = []

    def scrape_products(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        product_elements = soup.find_all('div', class_='mfo_card_content')

        for product in product_elements:
            title_div = product.find('div', class_='product_info')
            if title_div:
                title = title_div.find('h2').text.strip()
                clean_title_text = clean_title(title)
            else:
                clean_title_text = "未找到商品名稱"

            price_div = product.find('div', class_='price_info')
            if price_div:
                price = price_div.find('strong').text.strip().replace(',', '')  # 去掉逗號
                first_price = float(price.split()[0][1:])  # 只爬取第一個價格並轉為浮點數
            else:
                first_price = 0  # 沒有找到價格，設為0

            products.append((clean_title_text, first_price, '台灣大哥大'))

    # 單頁爬取
    url_single_page = 'https://www.myfone.com.tw/mfo/buy/cat/3263?pageNum=1'
    scrape_products(url_single_page)

    # 多頁爬取
    base_url = 'https://www.myfone.com.tw/mfo/buy/cat/34966?pageNum='
    for page_num in range(1, 27):  
        url = base_url + str(page_num)
        scrape_products(url)

    return products

# 創建手機資料庫
def create_mobile_db():
    conn = sqlite3.connect('products.db')
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS products (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            price REAL NOT NULL,
            store TEXT NOT NULL
        )
    ''')
    conn.commit()
    conn.close()

# 清空舊手機資料
def clear_mobile_db():
    conn = sqlite3.connect('products.db')
    c = conn.cursor()
    c.execute('DELETE FROM products')
    conn.commit()
    conn.close()

# 存入手機資料到資料庫
def save_mobile_to_db(products):
    conn = sqlite3.connect('products.db')
    c = conn.cursor()
    c.executemany('INSERT INTO products (name, price, store) VALUES (?, ?, ?)', products)
    conn.commit()
    conn.close()

# 搜尋手機資料
def search_mobile_product(product_name):
    conn = sqlite3.connect('products.db')
    c = conn.cursor()
    c.execute('SELECT name, price, store FROM products WHERE name LIKE ?', ('%' + product_name + '%',))
    results = c.fetchall()
    conn.close()
    return results

# 創建筆電資料庫
def create_laptop_db():
    conn = sqlite3.connect('laptops.db')
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS laptops (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            price REAL NOT NULL,
            store TEXT NOT NULL
        )
    ''')
    conn.commit()
    conn.close()

# 清空舊筆電資料
def clear_laptop_db():
    conn = sqlite3.connect('laptops.db')
    c = conn.cursor()
    c.execute('DELETE FROM laptops')
    conn.commit()
    conn.close()

# 存入筆電資料到資料庫
def save_laptop_to_db(products):
    conn = sqlite3.connect('laptops.db')
    c = conn.cursor()
    c.executemany('INSERT INTO laptops (name, price, store) VALUES (?, ?, ?)', products)
    conn.commit()
    conn.close()

# 搜尋筆電資料
def search_laptop_product(product_name):
    conn = sqlite3.connect('laptops.db')
    c = conn.cursor()
    c.execute('SELECT name, price, store FROM laptops WHERE name LIKE ?', ('%' + product_name + '%',))
    results = c.fetchall()
    conn.close()
    return results

# 創建手機資料庫
create_mobile_db()

# 清空舊手機資料
clear_mobile_db()

# 爬取傑昇通信資料並存入資料庫
products_jyes = fetch_data_from_jyes()
if products_jyes:
    save_mobile_to_db(products_jyes)
else:
    print("沒有從傑昇通信獲取到商品資料。")

# 爬取地標網通資料並存入資料庫，限制為 449 筆
products_landtop = fetch_data_from_landtop(limit=449)
if products_landtop:
    save_mobile_to_db(products_landtop)
else:
    print("沒有從地標網通獲取到商品資料。")

# 創建筆電資料庫
create_laptop_db()

# 清空舊筆電資料
clear_laptop_db()

# 爬取創捷國際資料並存入資料庫
products_teksource = fetch_data_from_teksource()
if products_teksource:
    save_laptop_to_db(products_teksource)
else:
    print("沒有從創捷國際獲取到商品資料。")

# 爬取台灣大哥大資料並存入資料庫
products_taiwanmobile = fetch_data_from_taiwanmobile()
if products_taiwanmobile:
    save_laptop_to_db(products_taiwanmobile)
else:
    print("沒有從台灣大哥大獲取到商品資料。")


ValueError: could not convert string to float: '$13290'

In [15]:
import os
import sqlite3
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage, TemplateSendMessage, ButtonsTemplate, PostbackTemplateAction, PostbackEvent

app = Flask(__name__)

# 設定 LINE Bot 的 Channel Access Token 和 Channel Secret
LINE_CHANNEL_ACCESS_TOKEN = os.getenv('LINE_ACCESS_TOKEN')
LINE_CHANNEL_SECRET = os.getenv('LINE_SECRET')

line_bot_api = LineBotApi(LINE_CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(LINE_CHANNEL_SECRET)

# 全域變數，用於暫存所選擇的商品資訊
awaiting_custom_query = False  # 用於標記是否在等待用戶輸入自定義查詢
selected_category = None  # 用於存儲所選擇的商品類型
selected_store = None  # 用於存儲所選擇的商店

# 設定Flask路由
@app.route("/", methods=['POST'])
def callback():
    signature = request.headers.get('X-Line-Signature', '')
    body = request.get_data(as_text=True)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

# 處理文字訊息事件
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    global awaiting_custom_query  # 使用全域變數

    if event.message.text == "預算":
        buttons_template = TemplateSendMessage(
            alt_text='預算選單',
            template=ButtonsTemplate(
                title='請選擇產品類型',
                text='請選擇您想要的產品類型',
                actions=[
                    PostbackTemplateAction(label='手機', data='action=choose_category&category=手機'),
                    PostbackTemplateAction(label='筆電', data='action=choose_category&category=筆電')
                ]
            )
        )
        line_bot_api.reply_message(event.reply_token, buttons_template)
    elif awaiting_custom_query:  # 如果等待用戶輸入自定義查詢
        query = event.message.text
        if query.lower() == "結束":
            awaiting_custom_query = False
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text='搜尋結束。'))
        else:
            send_custom_price_query_result(event.reply_token, query)

# 處理Postback事件
@handler.add(PostbackEvent)
def handle_postback(event):
    global awaiting_custom_query, selected_category, selected_store  # 使用全域變數

    data = event.postback.data

    if data.startswith('action=choose_category&category='):
        selected_category = data.split('=')[-1]
        if selected_category == '手機':
            buttons_template = TemplateSendMessage(
                alt_text='商店選單',
                template=ButtonsTemplate(
                    title='請選擇商店',
                    text='請選擇您想要的商店',
                    actions=[
                        PostbackTemplateAction(label='傑昇通信', data='action=choose_store&category=手機&store=傑昇通信'),
                        PostbackTemplateAction(label='地標網通', data='action=choose_store&category=手機&store=地標網通')
                    ]
                )
            )
        elif selected_category == '筆電':
            buttons_template = TemplateSendMessage(
                alt_text='商店選單',
                template=ButtonsTemplate(
                    title='請選擇商店',
                    text='請選擇您想要的商店',
                    actions=[
                        PostbackTemplateAction(label='創捷國際', data='action=choose_store&category=筆電&store=創捷國際'),
                        PostbackTemplateAction(label='台灣大哥大', data='action=choose_store&category=筆電&store=台灣大哥大')
                    ]
                )
            )
        line_bot_api.reply_message(event.reply_token, buttons_template)
    
    elif data.startswith('action=choose_store&category='):
        parts = data.split('&')
        selected_category = parts[1].split('=')[-1]
        selected_store = parts[2].split('=')[-1]

        if selected_category == '手機':
            price_ranges = ['10000以下', '10000-20000', '20000以上', '其他']
        elif selected_category == '筆電':
            price_ranges = ['30000以下', '30000-80000', '80000以上', '其他']

        actions = [PostbackTemplateAction(label=price, data=f'action=choose_price&category={selected_category}&store={selected_store}&price={price}') for price in price_ranges]

        buttons_template = TemplateSendMessage(
            alt_text='價格區間選單',
            template=ButtonsTemplate(
                title='請選擇價格區間',
                text='請選擇您想要的價格區間',
                actions=actions
            )
        )
        line_bot_api.reply_message(event.reply_token, buttons_template)
    
    elif data.startswith('action=choose_price&category='):
        parts = data.split('&')
        selected_category = parts[1].split('=')[-1]
        selected_store = parts[2].split('=')[-1]
        price = parts[3].split('=')[-1]

        if price == '其他':
            awaiting_custom_query = True  # 設置標記等待用戶輸入自定義查詢
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text='請輸入您想要的價格區間（格式：min-max），或輸入"結束"來結束搜尋：'))
        else:
            products = search_product_by_price(selected_category, selected_store, price)
            send_products_response(event.reply_token, products)

# 從資料庫中查詢產品
def search_product_by_price(category, store, price):
    if category == '手機':
        db_name = 'products.db'
        table_name = 'products'
    elif category == '筆電':
        db_name = 'products2.db'
        table_name = 'products2'
    
    conn = sqlite3.connect(db_name)
    c = conn.cursor()

    price_conditions = {
        '10000以下': "CAST(REPLACE(REPLACE(price, '$', ''), ',', '') AS INTEGER) < 10000",
        '10000-20000': "CAST(REPLACE(REPLACE(price, '$', ''), ',', '') AS INTEGER) BETWEEN 10000 AND 20000",
        '20000以上': "CAST(REPLACE(REPLACE(price, '$', ''), ',', '') AS INTEGER) > 20000",
        '30000以下': "CAST(REPLACE(REPLACE(price, '$', ''), ',', '') AS INTEGER) < 30000",
        '30000-80000': "CAST(REPLACE(REPLACE(price, '$', ''), ',', '') AS INTEGER) BETWEEN 30000 AND 80000",
        '80000以上': "CAST(REPLACE(REPLACE(price, '$', ''), ',', '') AS INTEGER) > 80000"
    }

    query = f"SELECT name, price, store FROM {table_name} WHERE store = ? AND {price_conditions[price]} AND price != '挑戰手機最低價'"
    c.execute(query, (store,))
    results = c.fetchall()
    conn.close()
    return results

def send_products_response(reply_token, products):
    if products:
        response_list = [f"產品名稱: {product[0]}\n價格: {product[1]}\n商店: {product[2]}" for product in products]
        # 確認回應不超過LINE API允許的最大長度（5000字元）
        response = "\n\n".join(response_list[:20])  # 每個回應最多包含20個產品資訊，以避免長度超過限制
    else:
        response = "找不到相關產品。"
    line_bot_api.reply_message(reply_token, TextSendMessage(text=response))

def send_custom_price_query_result(reply_token, query):
    try:
        min_price, max_price = map(int, query.split('-'))
        products = search_product_by_custom_price(selected_category, selected_store, min_price, max_price)
        send_products_response(reply_token, products)
    except ValueError:
        line_bot_api.reply_message(reply_token, TextSendMessage(text='格式錯誤。請輸入有效的價格區間（格式：min-max），或輸入"結束"來結束搜尋。'))

def search_product_by_custom_price(category, store, min_price, max_price):
    if category == '手機':
        db_name = 'products.db'
        table_name = 'products'
    elif category == '筆電':
        db_name = 'products2.db'
        table_name = 'products2'
    
    conn = sqlite3.connect(db_name)
    c = conn.cursor()

    query = f"SELECT name, price, store FROM {table_name} WHERE store = ? AND CAST(REPLACE(REPLACE(price, '$', ''), ',', '') AS INTEGER) BETWEEN ? AND ? AND price != '挑戰手機最低價'"
    c.execute(query, (store, min_price, max_price))
    results = c.fetchall()
    conn.close()
    return results

if __name__ == "__main__":
    app.run(port=5000)


 * Serving Flask app '__main__'


 * Debug mode: off


C:\Users\USER\AppData\Local\Temp\ipykernel_9020\2468096947.py:14: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(LINE_CHANNEL_ACCESS_TOKEN)
C:\Users\USER\AppData\Local\Temp\ipykernel_9020\2468096947.py:15: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(LINE_CHANNEL_SECRET)
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
C:\Users\USER\AppData\Local\Temp\ipykernel_9020\2468096947.py:51: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' 